# Классификация текста простыми методами

Загружем необходимые данные для nltk.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/ilkhom/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ilkhom/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /Users/ilkhom/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilkhom/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Мы будем использовать датасет fetch_20newsgroups. Он содержит коллекции новостей с 20 различных источников. Но мы возьмем только 4.

In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

In [3]:
set(newsgroups_train.target)

{0, 1, 2, 3}

Загружаем данные и таргеты

In [4]:
X_train = newsgroups_train.data
y_train = newsgroups_train.target

In [5]:
X_test = newsgroups_test.data
y_test = newsgroups_test.target

Смотрим на количество данных

In [6]:
len(X_train), len(y_train)

(2373, 2373)

In [7]:
len(X_test), len(y_test)

(1579, 1579)

TfidfVectorizer – это одновременно CountVectorizer после которого идет TfidfTransformer. 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_train_vec.shape

(2373, 38683)

Воспользуемся LogisticRegression.

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train_vec, y_train)

LogisticRegression()

Оценим модель.

In [10]:
X_test_vec = X_train_vec = vectorizer.transform(X_test)
y_pred = lr.predict(X_test_vec)

In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                 precision    recall  f1-score   support

      sci.crypt       0.95      0.90      0.93       396
sci.electronics       0.80      0.94      0.86       393
        sci.med       0.95      0.87      0.91       396
      sci.space       0.97      0.93      0.95       394

       accuracy                           0.91      1579
      macro avg       0.92      0.91      0.91      1579
   weighted avg       0.92      0.91      0.91      1579



## Предобработка данных

До этого мы не применяли предобработку. Посмотрим насколько она нам может помочь.

Рассмотрим сначала предобработку на одном примере.



Токенизируем.

In [13]:
x = X_train[0]
print(x)

From: al@escom.com (Al Donaldson)
Subject: Re: Once tapped, your code is no good any more.
Reply-To: al@escom.COM (Al Donaldson)
Organization: ESCOM Corp., Oakton VA (USA)
Distribution: na
Lines: 16

amolitor@nmsu.edu (Andrew Molitor) writes:
>Yes, those evil guys in the FBI can probably, with some
>effort, abuse the system. I got news for you, if the evil guys in
>the FBI decide they want to persecute you, they're gonna, ...

And if Richard Nixon had had this kind of toy, he wouldn't have had
to send people into the Watergate.

But that's not really the issue.  The real issue is whether this 
will be used to justify a ban against individuals' use of private 
(i.e., anything else) encryption methods.

Unrelated question...isn't the term "Clipper," as neat as it is,
already taken by Intergraph?

Al



In [14]:
x = x.lower()

In [15]:
x = nltk.word_tokenize(x)
print(x)

['from', ':', 'al', '@', 'escom.com', '(', 'al', 'donaldson', ')', 'subject', ':', 're', ':', 'once', 'tapped', ',', 'your', 'code', 'is', 'no', 'good', 'any', 'more', '.', 'reply-to', ':', 'al', '@', 'escom.com', '(', 'al', 'donaldson', ')', 'organization', ':', 'escom', 'corp.', ',', 'oakton', 'va', '(', 'usa', ')', 'distribution', ':', 'na', 'lines', ':', '16', 'amolitor', '@', 'nmsu.edu', '(', 'andrew', 'molitor', ')', 'writes', ':', '>', 'yes', ',', 'those', 'evil', 'guys', 'in', 'the', 'fbi', 'can', 'probably', ',', 'with', 'some', '>', 'effort', ',', 'abuse', 'the', 'system', '.', 'i', 'got', 'news', 'for', 'you', ',', 'if', 'the', 'evil', 'guys', 'in', '>', 'the', 'fbi', 'decide', 'they', 'want', 'to', 'persecute', 'you', ',', 'they', "'re", 'gon', 'na', ',', '...', 'and', 'if', 'richard', 'nixon', 'had', 'had', 'this', 'kind', 'of', 'toy', ',', 'he', 'would', "n't", 'have', 'had', 'to', 'send', 'people', 'into', 'the', 'watergate', '.', 'but', 'that', "'s", 'not', 'really', 't

Удалим слова со знаками препинания.

In [16]:
x = [word for word in x if word.isalnum()]
print(x)

['from', 'al', 'al', 'donaldson', 'subject', 're', 'once', 'tapped', 'your', 'code', 'is', 'no', 'good', 'any', 'more', 'al', 'al', 'donaldson', 'organization', 'escom', 'oakton', 'va', 'usa', 'distribution', 'na', 'lines', '16', 'amolitor', 'andrew', 'molitor', 'writes', 'yes', 'those', 'evil', 'guys', 'in', 'the', 'fbi', 'can', 'probably', 'with', 'some', 'effort', 'abuse', 'the', 'system', 'i', 'got', 'news', 'for', 'you', 'if', 'the', 'evil', 'guys', 'in', 'the', 'fbi', 'decide', 'they', 'want', 'to', 'persecute', 'you', 'they', 'gon', 'na', 'and', 'if', 'richard', 'nixon', 'had', 'had', 'this', 'kind', 'of', 'toy', 'he', 'would', 'have', 'had', 'to', 'send', 'people', 'into', 'the', 'watergate', 'but', 'that', 'not', 'really', 'the', 'issue', 'the', 'real', 'issue', 'is', 'whether', 'this', 'will', 'be', 'used', 'to', 'justify', 'a', 'ban', 'against', 'individuals', 'use', 'of', 'private', 'anything', 'else', 'encryption', 'methods', 'unrelated', 'question', 'is', 'the', 'term', '

Лемматизируем.

In [18]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/ilkhom/nltk_data...


True

In [19]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(word):
     tag = nltk.pos_tag([word])[0][1][0].upper()
     tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
     return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()
x = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in x]
print(x)

['from', 'al', 'al', 'donaldson', 'subject', 're', 'once', 'tapped', 'your', 'code', 'be', 'no', 'good', 'any', 'more', 'al', 'al', 'donaldson', 'organization', 'escom', 'oakton', 'va', 'usa', 'distribution', 'na', 'line', '16', 'amolitor', 'andrew', 'molitor', 'writes', 'yes', 'those', 'evil', 'guy', 'in', 'the', 'fbi', 'can', 'probably', 'with', 'some', 'effort', 'abuse', 'the', 'system', 'i', 'get', 'news', 'for', 'you', 'if', 'the', 'evil', 'guy', 'in', 'the', 'fbi', 'decide', 'they', 'want', 'to', 'persecute', 'you', 'they', 'gon', 'na', 'and', 'if', 'richard', 'nixon', 'have', 'have', 'this', 'kind', 'of', 'toy', 'he', 'would', 'have', 'have', 'to', 'send', 'people', 'into', 'the', 'watergate', 'but', 'that', 'not', 'really', 'the', 'issue', 'the', 'real', 'issue', 'be', 'whether', 'this', 'will', 'be', 'use', 'to', 'justify', 'a', 'ban', 'against', 'individual', 'use', 'of', 'private', 'anything', 'else', 'encryption', 'method', 'unrelated', 'question', 'be', 'the', 'term', 'cli

Удалим стоп-слова.

In [20]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
print(len(x))
x = [word for word in x if not word in stop_words]
print(x)
print(len(x))

131
['al', 'al', 'donaldson', 'subject', 'tapped', 'code', 'good', 'al', 'al', 'donaldson', 'organization', 'escom', 'oakton', 'va', 'usa', 'distribution', 'na', 'line', '16', 'amolitor', 'andrew', 'molitor', 'writes', 'yes', 'evil', 'guy', 'fbi', 'probably', 'effort', 'abuse', 'system', 'get', 'news', 'evil', 'guy', 'fbi', 'decide', 'want', 'persecute', 'gon', 'na', 'richard', 'nixon', 'kind', 'toy', 'would', 'send', 'people', 'watergate', 'really', 'issue', 'real', 'issue', 'whether', 'use', 'justify', 'ban', 'individual', 'use', 'private', 'anything', 'else', 'encryption', 'method', 'unrelated', 'question', 'term', 'clipper', 'neat', 'already', 'take', 'intergraph', 'al']
73


In [21]:
from tqdm import tqdm

def preprocces(X):
  X_proccess = []
  lemmatizer = WordNetLemmatizer()

  for x in tqdm(X):
    
    x = x.lower()
    x = nltk.word_tokenize(x)
    x = [word for word in x if word.isalnum()]
    x = [lemmatizer.lemmatize(w) for w in x]
    x = [word for word in x if not word in stop_words]
    X_proccess.append(' '.join(x))
  return X_proccess


X_train_proc = preprocces(X_train)
X_test_proc = preprocces(X_test)

100%|██████████| 1579/1579 [00:04<00:00, 361.93it/s]


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_proc)
X_test_vec = vectorizer.transform(X_test_proc)
print(X_train_vec.shape)
print(X_test_vec.shape)

(2373, 29448)
(1579, 29448)


In [23]:
lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                 precision    recall  f1-score   support

      sci.crypt       0.98      0.91      0.95       396
sci.electronics       0.84      0.97      0.90       393
        sci.med       0.96      0.92      0.94       396
      sci.space       0.98      0.95      0.96       394

       accuracy                           0.94      1579
      macro avg       0.94      0.94      0.94      1579
   weighted avg       0.94      0.94      0.94      1579



## Стемминг 

Воспользуемся стеммингом вместо лемматизации

In [24]:
from nltk.stem import PorterStemmer

def preprocces(X):
  X_proccess = []
  stemmer = PorterStemmer()

  for x in tqdm(X):
    
    x = x.lower()
    x = nltk.word_tokenize(x)
    x = [word for word in x if word.isalnum()]
    x = [stemmer.stem(w) for w in x]
    x = [word for word in x if not word in stop_words]
    X_proccess.append(' '.join(x))
  return X_proccess


X_train_proc = preprocces(X_train)
X_test_proc = preprocces(X_test)

100%|██████████| 1579/1579 [00:09<00:00, 174.84it/s]


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_proc)
X_test_vec = vectorizer.transform(X_test_proc)
print(X_train_vec.shape)
print(X_test_vec.shape)

(2373, 24082)
(1579, 24082)


In [26]:
lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                 precision    recall  f1-score   support

      sci.crypt       0.96      0.92      0.94       396
sci.electronics       0.84      0.95      0.89       393
        sci.med       0.95      0.91      0.93       396
      sci.space       0.99      0.94      0.96       394

       accuracy                           0.93      1579
      macro avg       0.93      0.93      0.93      1579
   weighted avg       0.93      0.93      0.93      1579

